## 4.1.	Violações dos padrões de qualidade ar no Brasil



### 4.1.1	Séries temporais das estações de monitoramento


**TENTAR FAZER UM DROPDOWN MENU PARA ESCOLHER QUAL SÉRIE TEMPORAL VISUALIZAR NO LIVRO. Colocar padrões da CONAMA**

ou Figura 5 – Perfil temporal das médias diárias de a) MP10, b) MP2,5 e c) O3 nas estações Volta da Capela (Barra Longa – MG) e Vila Moema (Tubarão – SC) no ano de 2023.


### 4.1.2	Estatísticas das estações de monitoramento 


**TENTAR FAZER UMA TABELA ITERATIVA ONDE O USUÁRIO POSSA FILTRAR AS ESTAÇÕES E NÚMERO DE VIOLAÇÕES**  
ou Tabela 10 – Ranqueamento das cinco estações com o maior número de violações dos Padrões de Qualidade do Ar de O 3, MP10, MP2,5 e SO2 no ano de 2023.

```{note}
Here is a note
```

```{warning}
Here is a warning
```

```{important}
Here is a important
```

```{caution}
Here is a caution
```

```{tip}
Here is a tip
```